In [1]:
!pip install torch torchvision
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html


Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html


Unregister dataset

In [3]:
from detectron2.data import MetadataCatalog, DatasetCatalog

# Unregister existing dataset if already registered
if "car_damage_train" in DatasetCatalog.list():
    DatasetCatalog.remove("car_damage_train")
    MetadataCatalog.remove("car_damage_train")

if "car_damage_test" in DatasetCatalog.list():
    DatasetCatalog.remove("car_damage_test")
    MetadataCatalog.remove("car_damage_test")


RegisterDataset

In [4]:
from detectron2.data.datasets import register_coco_instances

# Register the train and test datasets
register_coco_instances("car_damage_train", {}, "Dataset/train/_annotations.coco.json", "Dataset/train")
register_coco_instances("car_damage_test", {}, "Dataset/test/_annotations.coco.json", "Dataset/test")

car_damage_metadata = MetadataCatalog.get("car_damage_train")
dataset_dicts = DatasetCatalog.get("car_damage_train")



Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



Viewing some data 

In [5]:
import random
import cv2
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=car_damage_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2.imshow("Sample Image", vis.get_image()[:, :, ::-1])  # Corrected line
    cv2.waitKey(0)
    cv2.destroyAllWindows()  # Close the window after displaying the image


Configuring

In [6]:
import os
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("car_damage_train",)
cfg.DATASETS.TEST = ("car_damage_test",)
cfg.DATALOADER.NUM_WORKERS = 4  # Increase if your CPU has available cores
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 1  # Reduce batch size for CPU
cfg.SOLVER.BASE_LR = 0.0001  # Adjust learning rate for smaller batch size
cfg.SOLVER.MAX_ITER = 1000  # Reduce iterations if needed
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64  # Reduce batch size per image
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3

cfg.MODEL.DEVICE = "cpu"
cfg.OUTPUT_DIR = "./output"


Training 

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


If using GPU: Ensure that you have the correct CUDA-enabled PyTorch installed.
If using CPU: Modify the configuration to run on the CPU.

In [8]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

False
None


In [9]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[08/27 21:31:03 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

[08/27 21:31:04 d2.engine.train_loop]: Starting training from iteration 0


c:\Users\walte\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[08/27 21:33:04 d2.utils.events]:  eta: 1:33:21  iter: 19  total_loss: 2.303  loss_cls: 1.386  loss_box_reg: 0.3899  loss_rpn_cls: 0.1683  loss_rpn_loc: 0.02674    time: 5.8538  last_time: 5.7102  data_time: 0.1549  last_data_time: 0.0005   lr: 1.9981e-06  
[08/27 21:34:58 d2.utils.events]:  eta: 1:29:37  iter: 39  total_loss: 2.06  loss_cls: 1.364  loss_box_reg: 0.4817  loss_rpn_cls: 0.06189  loss_rpn_loc: 0.0127    time: 5.6955  last_time: 4.7619  data_time: 0.0006  last_data_time: 0.0006   lr: 3.9961e-06  
[08/27 21:36:53 d2.utils.events]:  eta: 1:29:25  iter: 59  total_loss: 2.159  loss_cls: 1.29  loss_box_reg: 0.6172  loss_rpn_cls: 0.03061  loss_rpn_loc: 0.01019    time: 5.7097  last_time: 5.2352  data_time: 0.0006  last_data_time: 0.0006   lr: 5.9941e-06  
[08/27 21:38:45 d2.utils.events]:  eta: 1:27:31  iter: 79  total_loss: 1.855  loss_cls: 1.162  loss_box_reg: 0.6739  loss_rpn_cls: 0.01408  loss_rpn_loc: 0.0109    time: 5.6818  last_time: 6.9907  data_time: 0.0006  last_data_t

In [7]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Load the configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # Ensure this matches the number of classes in your dataset

# Load the trained model weights
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.DEVICE = "cpu"  # Use CPU or "cuda" for GPU

# Set the model to inference mode
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set a threshold for predictions
predictor = DefaultPredictor(cfg)

# Now you can use the predictor for inference
# Example:
# outputs = predictor(im)  # where 'im' is your input image


c:\Users\walte\AppData\Local\Programs\Python\Python312\Lib\site-packages\fvcore\common\checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, 

TestingModel

In [8]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Initialize the evaluator
evaluator = COCOEvaluator("car_damage_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "car_damage_test")

# Perform the evaluation
metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
print(metrics)  # Print the evaluation metrics


COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

c:\Users\walte\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


KeyboardInterrupt: 

In [13]:
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
import cv2

# Load the trained model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # Path to the trained model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set a threshold for predictions
predictor = DefaultPredictor(cfg)

# Load an image for inference
image_path = "Dataset/test/severe_112_JPEG_jpg.rf.c2ff96a66830da8c1c96d9a11dba6302.jpg"
im = cv2.imread(image_path)

# Run the predictor
outputs = predictor(im)

# Visualize the predictions on the image
v = Visualizer(im[:, :, ::-1], metadata=car_damage_metadata, scale=0.8)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow("Prediction", v.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()


c:\Users\walte\AppData\Local\Programs\Python\Python312\Lib\site-packages\fvcore\common\checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, 